In [1]:
# library
from pyspark.sql.functions import udf, when, col, first, last, avg, lit
from pyspark.sql.types import BooleanType, IntegerType, StringType, DateType, FloatType
import pandas as pd
import numpy as np
from pyspark.sql import SparkSession
import time, datetime

# SparkSessjon 객체 생성 -> 모든 Spark 작업의 시작점
spark = SparkSession \
.builder \
.appName("Homework") \
.getOrCreate()

In [2]:
# Csv File Read
step_df = spark.read.load("/FileStore/tables/step_data.csv", format="csv", sep=",",  header = 'True', inferSchema='True')
weight_df = spark.read.load("/FileStore/tables/weight_data_float.csv", format="csv", sep=",",  header = 'True', inferSchema='True')
glucose_df = spark.read.load("/FileStore/tables/glucose_data_float.csv", format="csv", sep=",", header = 'True', inferSchema='True')


In [3]:
# data type check
step_df.printSchema()
weight_df.printSchema()
glucose_df.printSchema()

root
-- uid: integer (nullable = true)
-- day_time: long (nullable = true)
-- step_count: integer (nullable = true)
-- country_cd: string (nullable = true)
-- gender: string (nullable = true)
-- age_group: string (nullable = true)

root
-- uid: integer (nullable = true)
-- measurement_time: long (nullable = true)
-- weight: double (nullable = true)
-- country_cd: string (nullable = true)
-- gender: string (nullable = true)
-- age_group: string (nullable = true)

root
-- uid: integer (nullable = true)
-- measurement_time: long (nullable = true)
-- glucose: double (nullable = true)
-- country_cd: string (nullable = true)
-- gender: string (nullable = true)
-- age_group: string (nullable = true)

In [4]:
# data type 바꿔주는 함수 선언

# NA 값을 null로 변경하기
def cleanString(value):
  if(value == None) or ('null' in value):
    return None
  else:
    return value

# 유닉스시간을 Date Type으로 변경
def stringToDate(value):
  if value is None:
    return None
  else:
    return datetime.datetime.fromtimestamp(float(float(value)/float(1000)))


In [5]:
# 3개 dataframe의 attribute 형변환
cleanStringFunction = udf(cleanString, StringType())
cleanDateFunction = udf(stringToDate, DateType())

step_df = step_df \
    .withColumn(
      'day_time', cleanDateFunction('day_time'))\
    .withColumn(
      # null로 변경
      'age_group', cleanStringFunction('age_group')) \
    .withColumn(
      # null로 변경
      'gender', cleanStringFunction('gender'))

weight_df = weight_df \
    .withColumn(
      'measurement_time', cleanDateFunction('measurement_time'))\
    .withColumn(
      # null로 변경
      'gender', cleanStringFunction('gender'))

glucose_df = glucose_df \
    .withColumn(
      'measurement_time', cleanDateFunction('measurement_time'))\
    .withColumn(
      # null로 변경
      'gender', cleanStringFunction('gender'))


In [6]:
step_df.show()
weight_df.show()
glucose_df.show()

+---+----------+----------+----------+------+---------+
uid| day_time|step_count|country_cd|gender|age_group|
+---+----------+----------+----------+------+---------+
 1|2020-04-13| 9843| US| M| 10|
 2|2020-04-13| 3456| BR| M| 20|
 3|2020-04-13| 2346| KR| null| 30|
 4|2020-04-13| 7543| GB| F| 40|
 5|2020-04-13| 2345| DE| F| 50|
 6|2020-04-13| 11234| FR| null| null|
 7|2020-04-13| 14566| TR| F| 20|
 8|2020-04-13| 3345| RU| M| 30|
 9|2020-04-13| 3356| IN| M| 40|
 10|2020-04-13| 7754| KR| M| 50|
+---+----------+----------+----------+------+---------+

+---+----------------+-------+----------+------+---------+
uid|measurement_time| weight|country_cd|gender|age_group|
+---+----------------+-------+----------+------+---------+
 1| 2020-04-13| 76.0| US| M| 10|
 2| 2020-04-13|87.0898| BR| M| 20|
 3| 2020-04-13| 116.8| KR| null| null|
 4| 2020-04-12| 57.0| GB| F| 40|
 5| 2020-04-13|80.5581| DE| F| 50|
 6| 2020-04-13| 106.5| FR| null| 30|
 7| 2020-04-13| 92.0| TR| F| 20|
 8| 2020-04-13| 77.5| RU| M| 30|
 9| 2020-04-13|77.3829| IN| M| 40|
 10| 2020-04-13| 90.0| KR| M| 50|
+---+----------------+-------+----------+------+---------+

+---+----------------+---------+----------+------+---------+
uid|measurement_time| glucose|country_cd|gender|age_group|
+---+----------------+---------+----------+------+---------+
 1| 2020-04-13|7.0477247| US| M| 10|
 2| 2020-04-13| 7.769145| BR| M| 20|
 3| 2020-04-13|14.872364| KR| null| 30|
 4| 2020-04-12| 7.325194| GB| F| 40|
 5| 2020-04-13| 7.214206| DE| F| 50|
 6| 2020-04-13| 8.324084| FR| null| null|
 7| 2020-04-13| 5.216426| TR| F| 20|
 8| 2020-04-13|4.9944506| RU| M| 30|
 9| 2020-04-13| 7.321594| IN| M| 40|
 10| 2020-04-13|7.0200223| KR| M| 50|
+---+----------------+---------+----------+------+---------+

In [7]:
# Sanity Check
glucose_df = glucose_df.withColumn("glucose", when(col("glucose").isNull() , None).otherwise(col("glucose")))\
                         .withColumn("glucose", when(col("glucose") <= 0, None).otherwise(col("glucose")))\
                        .withColumn("glucose", when(col("glucose") <= 5, 5).otherwise(col("glucose")))\
                        .withColumn("glucose", when(col("glucose") >= 95, 95).otherwise(col("glucose")))

In [8]:
# 하루에 여러 번 측정이 가능한 데이터 하루치 평균으로 처리
weight_daily_df = weight_df.groupBy('uid','measurement_time')\
.mean('weight')\
.orderBy('uid')
weight_daily_df.show()

glucose_daily_df = glucose_df.groupBy('uid','measurement_time')\
.mean('glucose')\
.orderBy('uid')
glucose_daily_df.show()


+---+----------------+-----------+
uid|measurement_time|avg(weight)|
+---+----------------+-----------+
 1| 2020-04-13| 76.0|
 2| 2020-04-13| 87.0898|
 3| 2020-04-13| 116.8|
 4| 2020-04-12| 57.0|
 5| 2020-04-13| 80.5581|
 6| 2020-04-13| 106.5|
 7| 2020-04-13| 92.0|
 8| 2020-04-13| 77.5|
 9| 2020-04-13| 77.3829|
 10| 2020-04-13| 90.0|
+---+----------------+-----------+

+---+----------------+------------+
uid|measurement_time|avg(glucose)|
+---+----------------+------------+
 1| 2020-04-13| 7.0477247|
 2| 2020-04-13| 7.769145|
 3| 2020-04-13| 14.872364|
 4| 2020-04-12| 7.325194|
 5| 2020-04-13| 7.214206|
 6| 2020-04-13| 8.324084|
 7| 2020-04-13| 5.216426|
 8| 2020-04-13| 5.0|
 9| 2020-04-13| 7.321594|
 10| 2020-04-13| 7.0200223|
+---+----------------+------------+

# 과제1

In [10]:
# 과제 1 
# method 1 - 테이블을 uid당 각 데이터(step_count, weight, glucose) 전체 평균이라고 생각
step_mean_df = step_df.groupBy('uid', 'age_group').mean('step_count')
weight_mean_df = weight_df.groupBy('uid').mean('weight')
glucose_mean_df = glucose_df.groupBy('uid').mean('glucose')

joined_mean_df = step_mean_df.join(weight_mean_df, (step_mean_df['uid'] == weight_mean_df['uid']),)\
       .join(glucose_mean_df, (step_mean_df['uid'] == glucose_mean_df['uid']),)\
        .orderBy(step_mean_df['uid'])

joined_mean_aligned_df = joined_mean_df\
  .select(step_mean_df['avg(step_count)'].alias('step_count'), step_mean_df['age_group'],\
          weight_mean_df['avg(weight)'].alias('weight'), glucose_mean_df['avg(glucose)'].alias('glucose'))
joined_mean_aligned_df.show()


+----------+---------+-------+---------+
step_count|age_group| weight| glucose|
+----------+---------+-------+---------+
 9843.0| 10| 76.0|7.0477247|
 3456.0| 20|87.0898| 7.769145|
 2346.0| 30| 116.8|14.872364|
 7543.0| 40| 57.0| 7.325194|
 2345.0| 50|80.5581| 7.214206|
 11234.0| null| 106.5| 8.324084|
 14566.0| 20| 92.0| 5.216426|
 3345.0| 30| 77.5| 5.0|
 3356.0| 40|77.3829| 7.321594|
 7754.0| 50| 90.0|7.0200223|
+----------+---------+-------+---------+

In [11]:
# method 2 - uid로 세 테이블 단순 조인이라고 생각 
# 하루치 평균 테이블로 치환 후, 세 테이블 조인
joined_df = step_df.join(weight_daily_df, (step_df['uid'] == weight_daily_df['uid']),)\
       .join(glucose_daily_df, (step_df['uid'] == glucose_daily_df['uid']),)

joined_aligned_df = joined_df\
  .select(step_df['step_count'], step_df['age_group'], weight_daily_df['avg(weight)'].alias('weight'), glucose_daily_df['avg(glucose)'].alias('glucose'))

# Homework 1-1 Output 
joined_aligned_df.show()

+----------+---------+-------+---------+
step_count|age_group| weight| glucose|
+----------+---------+-------+---------+
 9843| 10| 76.0|7.0477247|
 3456| 20|87.0898| 7.769145|
 2346| 30| 116.8|14.872364|
 7543| 40| 57.0| 7.325194|
 2345| 50|80.5581| 7.214206|
 11234| null| 106.5| 8.324084|
 14566| 20| 92.0| 5.216426|
 3345| 30| 77.5| 5.0|
 3356| 40|77.3829| 7.321594|
 7754| 50| 90.0|7.0200223|
+----------+---------+-------+---------+

# 과제2

In [13]:
step_daily_df = step_df.select('uid','day_time','step_count')
step_daily_df.show()
weight_daily_df.show()
glucose_daily_df.show()

+---+----------+----------+
uid| day_time|step_count|
+---+----------+----------+
 1|2020-04-13| 9843|
 2|2020-04-13| 3456|
 3|2020-04-13| 2346|
 4|2020-04-13| 7543|
 5|2020-04-13| 2345|
 6|2020-04-13| 11234|
 7|2020-04-13| 14566|
 8|2020-04-13| 3345|
 9|2020-04-13| 3356|
 10|2020-04-13| 7754|
+---+----------+----------+

+---+----------------+-----------+
uid|measurement_time|avg(weight)|
+---+----------------+-----------+
 1| 2020-04-13| 76.0|
 2| 2020-04-13| 87.0898|
 3| 2020-04-13| 116.8|
 4| 2020-04-12| 57.0|
 5| 2020-04-13| 80.5581|
 6| 2020-04-13| 106.5|
 7| 2020-04-13| 92.0|
 8| 2020-04-13| 77.5|
 9| 2020-04-13| 77.3829|
 10| 2020-04-13| 90.0|
+---+----------------+-----------+

+---+----------------+------------+
uid|measurement_time|avg(glucose)|
+---+----------------+------------+
 1| 2020-04-13| 7.0477247|
 2| 2020-04-13| 7.769145|
 3| 2020-04-13| 14.872364|
 4| 2020-04-12| 7.325194|
 5| 2020-04-13| 7.214206|
 6| 2020-04-13| 8.324084|
 7| 2020-04-13| 5.216426|
 8| 2020-04-13| 5.0|
 9| 2020-04-13| 7.321594|
 10| 2020-04-13| 7.0200223|
+---+----------------+------------+

In [14]:
# uid list 얻기 
uid_list_df = step_daily_df.select('uid').distinct().union(\
weight_daily_df.select('uid').distinct()).union(\
glucose_daily_df.select('uid').distinct()).distinct().orderBy('uid')

In [15]:
step_df.show()
weight_df.show()

+---+----------+----------+----------+------+---------+
uid| day_time|step_count|country_cd|gender|age_group|
+---+----------+----------+----------+------+---------+
 1|2020-04-13| 9843| US| M| 10|
 2|2020-04-13| 3456| BR| M| 20|
 3|2020-04-13| 2346| KR| null| 30|
 4|2020-04-13| 7543| GB| F| 40|
 5|2020-04-13| 2345| DE| F| 50|
 6|2020-04-13| 11234| FR| null| null|
 7|2020-04-13| 14566| TR| F| 20|
 8|2020-04-13| 3345| RU| M| 30|
 9|2020-04-13| 3356| IN| M| 40|
 10|2020-04-13| 7754| KR| M| 50|
+---+----------+----------+----------+------+---------+

+---+----------------+-------+----------+------+---------+
uid|measurement_time| weight|country_cd|gender|age_group|
+---+----------------+-------+----------+------+---------+
 1| 2020-04-13| 76.0| US| M| 10|
 2| 2020-04-13|87.0898| BR| M| 20|
 3| 2020-04-13| 116.8| KR| null| null|
 4| 2020-04-12| 57.0| GB| F| 40|
 5| 2020-04-13|80.5581| DE| F| 50|
 6| 2020-04-13| 106.5| FR| null| 30|
 7| 2020-04-13| 92.0| TR| F| 20|
 8| 2020-04-13| 77.5| RU| M| 30|
 9| 2020-04-13|77.3829| IN| M| 40|
 10| 2020-04-13| 90.0| KR| M| 50|
+---+----------------+-------+----------+------+---------+

In [16]:
info_df = step_df.join(weight_df, (step_df['uid'] == weight_df['uid'])  ,'full_outer' )\
  .join(glucose_df, (step_df['uid'] == glucose_df['uid'])  ,'full_outer' )\
.orderBy(step_df['uid'])
info_df.show()

+---+----------+----------+----------+------+---------+---+----------------+-------+----------+------+---------+---+----------------+---------+----------+------+---------+
uid| day_time|step_count|country_cd|gender|age_group|uid|measurement_time| weight|country_cd|gender|age_group|uid|measurement_time| glucose|country_cd|gender|age_group|
+---+----------+----------+----------+------+---------+---+----------------+-------+----------+------+---------+---+----------------+---------+----------+------+---------+
 1|2020-04-13| 9843| US| M| 10| 1| 2020-04-13| 76.0| US| M| 10| 1| 2020-04-13|7.0477247| US| M| 10|
 2|2020-04-13| 3456| BR| M| 20| 2| 2020-04-13|87.0898| BR| M| 20| 2| 2020-04-13| 7.769145| BR| M| 20|
 3|2020-04-13| 2346| KR| null| 30| 3| 2020-04-13| 116.8| KR| null| null| 3| 2020-04-13|14.872364| KR| null| 30|
 4|2020-04-13| 7543| GB| F| 40| 4| 2020-04-12| 57.0| GB| F| 40| 4| 2020-04-12| 7.325194| GB| F| 40|
 5|2020-04-13| 2345| DE| F| 50| 5| 2020-04-13|80.5581| DE| F| 50| 5| 2020-04-13| 7.214206| DE| F| 50|
 6|2020-04-13| 11234| FR| null| null| 6| 2020-04-13| 106.5| FR| null| 30| 6| 2020-04-13| 8.324084| FR| null| null|
 7|2020-04-13| 14566| TR| F| 20| 7| 2020-04-13| 92.0| TR| F| 20| 7| 2020-04-13| 5.216426| TR| F| 20|
 8|2020-04-13| 3345| RU| M| 30| 8| 2020-04-13| 77.5| RU| M| 30| 8| 2020-04-13| 5.0| RU| M| 30|
 9|2020-04-13| 3356| IN| M| 40| 9| 2020-04-13|77.3829| IN| M| 40| 9| 2020-04-13| 7.321594| IN| M| 40|
 10|2020-04-13| 7754| KR| M| 50| 10| 2020-04-13| 90.0| KR| M| 50| 10| 2020-04-13|7.0200223| KR| M| 50|
+---+----------+----------+----------+------+---------+---+----------------+-------+----------+------+---------+---+----------------+---------+----------+------+---------+

In [17]:
info_df = info_df.select(step_df['uid'].alias('step_uid'),weight_df['uid'].alias('weight_uid'),glucose_df['uid'].alias('glucose_uid'),\
              step_df['country_cd'].alias('step_country_cd'), weight_df['country_cd'].alias('weight_country_cd'),glucose_df['country_cd'].alias('glucose_country_cd'),\
              step_df['gender'].alias('step_gender'), weight_df['gender'].alias('weight_gender'),glucose_df['gender'].alias('glucose_gender'),\
              step_df['age_group'].alias('step_age_group'), weight_df['age_group'].alias('weight_age_group'),glucose_df['age_group'].alias('glucose_age_group'))

In [18]:
# Imputation using 3 tables
# step table을 기준으로 missing data를 채워준다
info_imputing_df = info_df.withColumn("step_uid", when(col("step_uid").isNull(),  when(col("weight_uid").isNull(), col("glucose_uid") ).otherwise(col("weight_uid")) ).otherwise(col("step_uid")))\
.withColumn("step_gender", when(col("step_gender").isNull(),  when(col("weight_gender").isNull(), col("glucose_gender") ).otherwise(col("weight_gender")) ).otherwise(col("step_gender")))\
.withColumn("step_country_cd", when(col("step_country_cd").isNull(),  when(col("weight_country_cd").isNull(), col("glucose_country_cd") ).otherwise(col("weight_country_cd")) ).otherwise(col("step_country_cd")))\
.withColumn("step_age_group", when(col("step_age_group").isNull(),  when(col("weight_age_group").isNull(), col("glucose_age_group") ).otherwise(col("weight_age_group")) ).otherwise(col("step_age_group")))
info_imputing_df.show()

+--------+----------+-----------+---------------+-----------------+------------------+-----------+-------------+--------------+--------------+----------------+-----------------+
step_uid|weight_uid|glucose_uid|step_country_cd|weight_country_cd|glucose_country_cd|step_gender|weight_gender|glucose_gender|step_age_group|weight_age_group|glucose_age_group|
+--------+----------+-----------+---------------+-----------------+------------------+-----------+-------------+--------------+--------------+----------------+-----------------+
 1| 1| 1| US| US| US| M| M| M| 10| 10| 10|
 2| 2| 2| BR| BR| BR| M| M| M| 20| 20| 20|
 3| 3| 3| KR| KR| KR| null| null| null| 30| null| 30|
 4| 4| 4| GB| GB| GB| F| F| F| 40| 40| 40|
 5| 5| 5| DE| DE| DE| F| F| F| 50| 50| 50|
 6| 6| 6| FR| FR| FR| null| null| null| 30| 30| null|
 7| 7| 7| TR| TR| TR| F| F| F| 20| 20| 20|
 8| 8| 8| RU| RU| RU| M| M| M| 30| 30| 30|
 9| 9| 9| IN| IN| IN| M| M| M| 40| 40| 40|
 10| 10| 10| KR| KR| KR| M| M| M| 50| 50| 50|
+--------+----------+-----------+---------------+-----------------+------------------+-----------+-------------+--------------+--------------+----------------+-----------------+

In [19]:
info_imputed_df = info_imputing_df\
.select(info_imputing_df.step_uid.alias('uid'),\
        info_imputing_df.step_country_cd.alias('country_cd'),\
        info_imputing_df.step_gender.alias('gender'),\
        info_imputing_df.step_age_group.alias( 'age_group'))\
.orderBy('uid')
info_imputed_df.show()

+---+----------+------+---------+
uid|country_cd|gender|age_group|
+---+----------+------+---------+
 1| US| M| 10|
 2| BR| M| 20|
 3| KR| null| 30|
 4| GB| F| 40|
 5| DE| F| 50|
 6| FR| null| 30|
 7| TR| F| 20|
 8| RU| M| 30|
 9| IN| M| 40|
 10| KR| M| 50|
+---+----------+------+---------+

In [20]:
# uid list data와 country_cd, gender, age_group 조인
res1 = uid_list_df.join(info_imputed_df, uid_list_df['uid'] == info_imputed_df['uid'])\
.drop(info_imputed_df['uid'])

In [21]:
# Data Reshaping

# 이미 weight,glucose table의 데이터를 일일 평균 데이터로 치환했기에 avg말고 first를 사용했습니다.
step_reshaped_df = step_daily_df.groupBy('uid').pivot("day_time").agg(first(col("step_count")))
step_reshaped_df = step_reshaped_df.withColumnRenamed("uid", "uid_s").withColumnRenamed("2020-04-13", "2020-04-13_s")
step_reshaped_df.show()
weight_reshaped_df = weight_daily_df.groupBy('uid').pivot("measurement_time").agg(first(col("avg(weight)")))
weight_reshaped_df = weight_reshaped_df.withColumnRenamed("uid", "uid_w").withColumnRenamed("2020-04-12", "2020-04-12_w")\
       .withColumnRenamed("2020-04-13", "2020-04-13_w")
weight_reshaped_df.show()
glucose_reshaped_df = glucose_daily_df.groupBy('uid').pivot("measurement_time").agg(first(col("avg(glucose)")*18))
glucose_reshaped_df = glucose_reshaped_df.withColumnRenamed("uid", "uid_g").withColumnRenamed("2020-04-12", "2020-04-12_g")\
       .withColumnRenamed("2020-04-13", "2020-04-13_g")
glucose_reshaped_df.show()

+-----+------------+
uid_s|2020-04-13_s|
+-----+------------+
 1| 9843|
 6| 11234|
 3| 2346|
 5| 2345|
 9| 3356|
 4| 7543|
 8| 3345|
 7| 14566|
 10| 7754|
 2| 3456|
+-----+------------+

+-----+------------+------------+
uid_w|2020-04-12_w|2020-04-13_w|
+-----+------------+------------+
 1| null| 76.0|
 2| null| 87.0898|
 3| null| 116.8|
 4| 57.0| null|
 5| null| 80.5581|
 6| null| 106.5|
 7| null| 92.0|
 8| null| 77.5|
 9| null| 77.3829|
 10| null| 90.0|
+-----+------------+------------+

+-----+------------+------------------+
uid_g|2020-04-12_g| 2020-04-13_g|
+-----+------------+------------------+
 1| null| 126.8590446|
 2| null| 139.84461|
 3| null|267.70255199999997|
 4| 131.853492| null|
 5| null| 129.855708|
 6| null|149.83351199999998|
 7| null| 93.895668|
 8| null| 90.0|
 9| null| 131.788692|
 10| null| 126.3604014|
+-----+------------+------------------+

In [22]:
health_data_df = step_reshaped_df\
.join(weight_reshaped_df,step_reshaped_df['uid_s']==weight_reshaped_df['uid_w'],'outer')\
.join(glucose_reshaped_df,step_reshaped_df['uid_s']==glucose_reshaped_df['uid_g'],'outer')\
.orderBy(step_reshaped_df['uid_s'])\
.drop(*['uid_w','uid_g'])

In [23]:
res2 = res1.join(health_data_df, health_data_df['uid_s']== res1['uid'])\
.drop('uid_s')

# Homework 2
res2.show()

+---+----------+------+---------+------------+------------+------------+------------+------------------+
uid|country_cd|gender|age_group|2020-04-13_s|2020-04-12_w|2020-04-13_w|2020-04-12_g| 2020-04-13_g|
+---+----------+------+---------+------------+------------+------------+------------+------------------+
 1| US| M| 10| 9843| null| 76.0| null| 126.8590446|
 2| BR| M| 20| 3456| null| 87.0898| null| 139.84461|
 3| KR| null| 30| 2346| null| 116.8| null|267.70255199999997|
 4| GB| F| 40| 7543| 57.0| null| 131.853492| null|
 5| DE| F| 50| 2345| null| 80.5581| null| 129.855708|
 6| FR| null| 30| 11234| null| 106.5| null|149.83351199999998|
 7| TR| F| 20| 14566| null| 92.0| null| 93.895668|
 8| RU| M| 30| 3345| null| 77.5| null| 90.0|
 9| IN| M| 40| 3356| null| 77.3829| null| 131.788692|
 10| KR| M| 50| 7754| null| 90.0| null| 126.3604014|
+---+----------+------+---------+------------+------------+------------+------------+------------------+